# BSPF Analysis - Statistics Plots

In [1]:
import os
import pandas as pd
import numpy as np
import obspy as obs

import matplotlib.pyplot as plt

In [2]:
from functions.compute_orthogonal_distance_regression import __compute_orthogonal_distance_regression
from functions.compute_linear_regression import __compute_linear_regression

In [3]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

## Configurations

In [4]:
config = {}

config['path_to_data'] = data_path+"BSPF/data/"

config['path_to_figs'] = data_path+"BSPF/figures/"

config['path_to_catalogs'] = data_path+"BSPF/data/catalogs/"

config['catalog'] = ""

config['translation_type'] = "ACC"

## Load Data

In [5]:
data_amax = pd.read_pickle(config['path_to_data']+f"bspf_analysisdata_amax_{config['translation_type']}.pkl")

data_snr = pd.read_pickle(config['path_to_data']+f"bspf_analysisdata_snr_{config['translation_type']}.pkl")

data_adr = pd.read_pickle(config['path_to_data']+f"bspf_analysisdata_adr_{config['translation_type']}.pkl")

data_adr_cc = pd.read_pickle(config['path_to_data']+f"bspf_analysisdata_adr_cc_{config['translation_type']}.pkl")


data_good = pd.read_pickle(config['path_to_catalogs']+"good_events_20221001_20230930.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '/home/andbro/kilauea-data/BSPF/data/bspf_analysisdata_amax_ACC.pkl'

In [ ]:
print(f" AMAX:  all: {data_amax.origin.size} no_nan: {data_amax.origin.dropna().size}")
print(f" SNR:  all: {data_snr.origin.size} no_nan: {data_snr.origin.dropna().size}")
print(f" ADR:  all: {data_adr.origin.size} no_nan: {data_adr.origin.dropna().size}")
print(f" CC:  all: {data_adr_cc.origin.size} no_nan: {data_adr_cc.origin.dropna().size}")

# data_amax = data_amax.dropna()
# data_snr = data_snr.dropna()
# data_adr = data_adr.dropna()

In [ ]:
# data_adr.nunique()
data_adr = data_adr.dropna()

## Modify Data

Filter data for events with coincidence sum 6

In [ ]:
data_amax_cosum_6 = data_amax[data_amax.CoincidenceSum == 6]

data_snr_cosum_6 = data_snr[data_snr.CoincidenceSum == 6]

data_adr_cosum_6 = data_adr[data_adr.CoincidenceSum == 6]

data_adr_cc_cosum_6 = data_adr_cc[data_adr_cc.CoincidenceSum == 6]

In [ ]:
print(f" AMAX:  {data_amax_cosum_6.origin.size}")
print(f" SNR:   {data_snr_cosum_6.origin.size}")
print(f" ADR:   {data_adr_cosum_6.origin.size}")
print(f" CC:   {data_adr_cc_cosum_6.origin.size}")

## Plots

In [ ]:
# max_rot = []
# for nn in range(data_amax_cosum_6.shape[0]):
#     max_rot.append(np.nanmax([data_amax_cosum_6.iloc[nn].BSPF__E_amax, data_amax_cosum_6.iloc[nn].BSPF__N_amax, data_amax_cosum_6.iloc[nn].BSPF__Z_amax]))

max_rot = []
for nn in range(data_amax_cosum_6.shape[0]):
    max_rot.append(np.sqrt(data_amax_cosum_6.iloc[nn].BSPF__E_amax**2 + data_amax_cosum_6.iloc[nn].BSPF__N_amax**2 + data_amax_cosum_6.iloc[nn].BSPF__Z_amax**2))


In [ ]:
def __makeplot(data1, data2, data3, data4, max_rot):

    import matplotlib

    fig, ax = plt.subplots(1,1,figsize=(15,5))

    font=14

    cmap = plt.get_cmap("viridis", 6)

    # dat = np.log10(data4.BSPF__N_snr)
    # dat = np.log10(data4.PFO_10_Z_snr)

    dat = np.log10(max_rot)

    ax.scatter(data1.Hdistance_km, data1.Magnitude, s=30, color="grey", zorder=1, label=f"all (N={len(data1)})")

    cax = ax.scatter(data2.Hdistance_km, data2.Magnitude, s=50, lw=0.5, edgecolors="k",
                     c=dat, cmap=cmap, zorder=2, label=f"co.sum 6 (N={len(data2)})", vmin=-7, vmax=-4)

    # cax = ax.scatter(data_good.Hdistance_km, data_good.Magnitude, s=50, lw=0.5, edgecolors="r",
    #                  c=dat, cmap=cmap, zorder=2, label=f"co.sum 6 (N={len(data2)})", vmin=-7, vmax=-4)


    ax.grid(ls="--", color="grey", alpha=0.6, zorder=0,which="both")
    ax.legend(loc=4, fontsize=font)

    ax.set_ylabel("magnitude", fontsize=font)
    ax.set_xlabel("hypocentral distance (km)", fontsize=font)

    ax.set_xscale("log")
    ax.set_xlim(left=7)

    cbar = plt.colorbar(cax, pad=0.02)
    # cbar.set_label(r"log$_{10}$(max. rotation rate) (rad/s)", fontsize=font)
    cbar.set_label(r"log$_{10}$($\sqrt{N^2 + E^2 + Z^2}$) (rad/s)", fontsize=font)
#     cbar.set_clim(0,4)

    ax.set_xticks([10, 20, 30, 40, 50, 100])
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.show()
    return fig

In [ ]:
fig = __makeplot(data_amax, data_amax_cosum_6, data_snr, data_snr_cosum_6, max_rot)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_mag_hdist.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot2(data1, data2, data3):

    import matplotlib

    max_rot1 = []
    for nn in range(data1.shape[0]):
        max_rot1.append(np.sqrt(data1.iloc[nn].BSPF__E_amax**2 + data1.iloc[nn].BSPF__N_amax**2 + data1.iloc[nn].BSPF__Z_amax**2))
    max_rot2 = []
    for nn in range(data2.shape[0]):
        max_rot2.append(np.sqrt(data2.iloc[nn].BSPF__E_amax**2 + data2.iloc[nn].BSPF__N_amax**2 + data2.iloc[nn].BSPF__Z_amax**2))
    max_rot3 = []
    for nn in range(data3.shape[0]):
        max_rot3.append(np.sqrt(data3.iloc[nn].BSPF__E_amax**2 + data3.iloc[nn].BSPF__N_amax**2 + data3.iloc[nn].BSPF__Z_amax**2))

    vmin, vmax = min([min(max_rot1), min(max_rot2), min(max_rot3)]), max([max(max_rot1), max(max_rot2), max(max_rot3)])
    vmin, vmax = -8.0, -4.5

    fig, ax = plt.subplots(1, 1, figsize=(15, 5))

    font = 14

    cmap = plt.get_cmap("viridis", 7)


    cax = ax.scatter(data1.Hdistance_km, data1.Magnitude, s=30, marker="o", c=np.log10(max_rot1), cmap=cmap, zorder=2, 
                     label=f"CoIn. 4 (N={len(data1)})", vmin=vmin, vmax=vmax, edgecolors="black")

    ax.scatter(data2.Hdistance_km, data2.Magnitude, s=30, marker="^", c=np.log10(max_rot2), cmap=cmap, zorder=2, 
               label=f"CoIn. 5 (N={len(data2)})", vmin=vmin, vmax=vmax, edgecolors="black")
    ax.scatter(data3.Hdistance_km, data3.Magnitude, s=30, marker="s", c=np.log10(max_rot3), cmap=cmap, zorder=2, 
               label=f"CoIn. 6 (N={len(data3)})", vmin=vmin, vmax=vmax, edgecolors="black")


    ax.grid(ls="--", color="grey", alpha=0.6, zorder=0, which="both")
    ax.legend(loc=4, fontsize=font-1)

    ax.set_ylabel("Magnitude", fontsize=font)
    ax.set_xlabel("Hypocentral Distance (km)", fontsize=font)

    ax.set_xscale("log")
    ax.set_xlim(left=7)

    cbar = plt.colorbar(cax, pad=0.02)
    # cbar.set_label(r"log$_{10}$(max. rotation rate) (rad/s)", fontsize=font)
    cbar.set_label(r"log$_{10}$($\sqrt{N^2 + E^2 + Z^2}$) (rad/s)", fontsize=font)
#     cbar.set_clim(0,4)

    ax.set_xticks([10, 20, 30, 40, 50, 100])
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.show()
    return fig

In [ ]:
data1 = data_amax[data_amax.CoincidenceSum == 4]
data2 = data_amax[data_amax.CoincidenceSum == 5]
data3 = data_amax[data_amax.CoincidenceSum == 6]


fig = __makeplot2(data1, data2, data3)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_mag_hdist2.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

## Polar Plot

In [ ]:
def __polar_plot(data1, data2):

    r1 = data1.Edistance_km
    r2 = data2.Edistance_km
    theta1 = np.radians(data1.BAZ)
    theta2 = np.radians(data2.BAZ)


    import matplotlib
    cmap = matplotlib.colors.ListedColormap(["tab:red", "tab:orange", "tab:green"])


    data1.sort_values(by="Magnitude")

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))

    ax.set_theta_offset(np.pi/2)
    ax.set_theta_direction(-1)

    ax.scatter(theta1, r1, s=4**data1.Magnitude, c=data_amax.CoincidenceSum, cmap=cmap, zorder=2, edgecolors="k", alpha=0.7)
    # ax.scatter(theta2, r2, s=4**data2.Magnitude, lw=0.5, edgecolors="k", zorder=3)

    ## station
    ax.scatter(0, 0, zorder=3, s=100, marker='^', c="r", lw=0.8, edgecolors="k")

#     ax.set_rmax(2)

#     ax.set_rticks([0.5, 1, 1.5, 2])  # Less radial ticks
    ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
    ax.grid(True, zorder=0)

    ax.set_title("", va='bottom')
    plt.show()
    return fig

In [ ]:
fig = __polar_plot(data_amax, data_snr)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_polar.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

## Amplitude Maxima

In [ ]:
def __makeplot_amax(data1, data2, xvalue):

    NJ = 4
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10))

    font = 14

    scale = 1
    msize = 12
    linew = 0.5

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.PFO_10_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,0].scatter(data2[xvalue], data2.PFO_10_Z_amax*scale, s=msize, color="tab:red", label="XPFO Z", lw=linew, edgecolors="k")

    ax[0,1].scatter(data1[xvalue], data1.PFO_10_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,1].scatter(data2[xvalue], data2.PFO_10_N_amax*scale, s=msize, color="tab:red", label="XPFO N", lw=linew, edgecolors="k")

    ax[0,2].scatter(data1[xvalue], data1.PFO_10_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,2].scatter(data2[xvalue], data2.PFO_10_E_amax*scale, s=msize, color="tab:red", label="XPFO E", lw=linew, edgecolors="k")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_amax*scale, s=msize, color="tab:red", label="ADR all Z", lw=linew, edgecolors="k")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_amax*scale, s=msize, color="tab:red", label="ADR all N", lw=linew, edgecolors="k")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_amax*scale, s=msize, color="tab:red", label="ADR all E", lw=linew, edgecolors="k")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_in_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,0].scatter(data2[xvalue], data2.RPFO_in_Z_amax*scale, s=msize, color="tab:red", label="ADR inner Z", lw=linew, edgecolors="k")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_in_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,1].scatter(data2[xvalue], data2.RPFO_in_N_amax*scale, s=msize, color="tab:red", label="ADR inner N", lw=linew, edgecolors="k")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_in_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,2].scatter(data2[xvalue], data2.RPFO_in_E_amax*scale, s=msize, color="tab:red", label="ADR inner E", lw=linew, edgecolors="k")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.BSPF__Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,0].scatter(data2[xvalue], data2.BSPF__Z_amax*scale, s=msize, color="tab:red", label="BSPF Z", lw=linew, edgecolors="k")

    ax[3,1].scatter(data1[xvalue], data1.BSPF__N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,1].scatter(data2[xvalue], data2.BSPF__N_amax*scale, s=msize, color="tab:red", label="BSPF N", lw=linew, edgecolors="k")

    ax[3,2].scatter(data1[xvalue], data1.BSPF__E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,2].scatter(data2[xvalue], data2.BSPF__E_amax*scale, s=msize, color="tab:red", label="BSPF E", lw=linew, edgecolors="k")



    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            # ax[i,j].set_xscale("log")
            ax[i,j].legend(loc=4)
            ax[3,j].set_xlabel(xvalue, fontsize=font)
            ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    for j in range(NI):
        ax[0,j].set_ylim(8e-7, 5e-2)
        ax[1,j].set_ylim(1e-11, 5e-5)
        ax[2,j].set_ylim(1e-10, 5e-5)
        ax[3,j].set_ylim(1e-8, 1e-4)

    ax[0,0].set_ylabel(r"Acc. (m/s$^2$)", fontsize=font)
    ax[1,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[2,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[3,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)

    plt.show()
    return fig

In [ ]:
fig = __makeplot_amax(data_amax, data_amax_cosum_6, "Magnitude")

fig.savefig(config['path_to_figs']+f"BSPF_statistics_ampl_mag.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot_amax_colored(data1, data2, xvalue):

    NJ = 4
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10))

    font = 14

    scale = 1
    msize = 12
    linew = 0.1

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.PFO_10_Z_amax*scale, s=msize, color="grey", alpha=0.7, label="XPFO Z")
    # ax[0,0].scatter(data2[xvalue], data2.PFO_10_Z_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[0,1].scatter(data1[xvalue], data1.PFO_10_N_amax*scale, s=msize, color="grey", alpha=0.7, label="XPFO N")
    # ax[0,1].scatter(data2[xvalue], data2.PFO_10_N_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[0,2].scatter(data1[xvalue], data1.PFO_10_E_amax*scale, s=msize, color="grey", alpha=0.7, label="XPFO E")
    # ax[0,2].scatter(data2[xvalue], data2.PFO_10_E_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR all Z")
    # ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR all N")
    # ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR all E")
    # ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_in_Z_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR inner Z")
    # ax[2,0].scatter(data2[xvalue], data2.RPFO_in_Z_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_in_N_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR inner N")
    # ax[2,1].scatter(data2[xvalue], data2.RPFO_in_N_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_in_E_amax*scale, s=msize, color="grey", alpha=0.7, label="ADR inner E")
    # ax[2,2].scatter(data2[xvalue], data2.RPFO_in_E_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.BSPF__Z_amax*scale, s=msize, color="grey", alpha=0.7, label="BSPF Z")
    # ax[3,0].scatter(data2[xvalue], data2.BSPF__Z_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[3,1].scatter(data1[xvalue], data1.BSPF__N_amax*scale, s=msize, color="grey", alpha=0.7, label="BSPF N")
    # ax[3,1].scatter(data2[xvalue], data2.BSPF__N_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")

    ax[3,2].scatter(data1[xvalue], data1.BSPF__E_amax*scale, s=msize, color="grey", alpha=0.7, label="BSPF E")
    # ax[3,2].scatter(data2[xvalue], data2.BSPF__E_amax*scale, s=msize, color="red", lw=linew, edgecolors="k")



    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            # ax[i,j].set_xscale("log")
            ax[i,j].legend(loc=4)
            ax[3,j].set_xlabel(xvalue, fontsize=font)
            ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    for j in range(NI):
        ax[0,j].set_ylim(8e-7, 5e-2)
        ax[1,j].set_ylim(1e-11, 5e-5)
        ax[2,j].set_ylim(1e-10, 5e-5)
        ax[3,j].set_ylim(1e-8, 1e-4)

    ax[0,0].set_ylabel(r"Acc. (m/s$^2$)", fontsize=font)
    ax[1,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[2,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[3,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)

    plt.show()
    return fig

In [ ]:
# fig = __makeplot_amax_colored(data_amax, data_amax_cosum_6, "Magnitude")

# fig.savefig(config['path_to_figs']+f"BSPF_statistics_ampl_mag_colored.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot_amax2(data1, data2, xvalue):

    NJ = 4
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10))

    font = 14

    scale = 1
    msize = 12
    linew = 0.1

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.PFO_10_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,0].scatter(data2[xvalue], data2.PFO_10_Z_amax*scale, s=msize, color="red", label="XPFO Z", lw=linew, edgecolors="k")

    ax[0,1].scatter(data1[xvalue], data1.PFO_10_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,1].scatter(data2[xvalue], data2.PFO_10_N_amax*scale, s=msize, color="red", label="XPFO N", lw=linew, edgecolors="k")

    ax[0,2].scatter(data1[xvalue], data1.PFO_10_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[0,2].scatter(data2[xvalue], data2.PFO_10_E_amax*scale, s=msize, color="red", label="XPFO E", lw=linew, edgecolors="k")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_amax*scale, s=msize, color="red", label="ADR all Z", lw=linew, edgecolors="k")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_amax*scale, s=msize, color="red", label="ADR all N", lw=linew, edgecolors="k")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_amax*scale, s=msize, color="red", label="ADR all E", lw=linew, edgecolors="k")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_in_Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,0].scatter(data2[xvalue], data2.RPFO_in_Z_amax*scale, s=msize, color="red", label="ADR inner Z", lw=linew, edgecolors="k")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_in_N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,1].scatter(data2[xvalue], data2.RPFO_in_N_amax*scale, s=msize, color="red", label="ADR inner N", lw=linew, edgecolors="k")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_in_E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[2,2].scatter(data2[xvalue], data2.RPFO_in_E_amax*scale, s=msize, color="red", label="ADR inner E", lw=linew, edgecolors="k")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.BSPF__Z_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,0].scatter(data2[xvalue], data2.BSPF__Z_amax*scale, s=msize, color="red", label="BSPF Z", lw=linew, edgecolors="k")

    ax[3,1].scatter(data1[xvalue], data1.BSPF__N_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,1].scatter(data2[xvalue], data2.BSPF__N_amax*scale, s=msize, color="red", label="BSPF N", lw=linew, edgecolors="k")

    ax[3,2].scatter(data1[xvalue], data1.BSPF__E_amax*scale, s=msize, color="grey", alpha=0.7)
    ax[3,2].scatter(data2[xvalue], data2.BSPF__E_amax*scale, s=msize, color="red", label="BSPF E", lw=linew, edgecolors="k")


    # for j in range(1,3):
    #     ax[j,0].set_ylim(1e-9, 1e-5)
    #     ax[j,1].set_ylim(1e-9, 1e-5)
    #     ax[j,2].set_ylim(1e-9, 1e-5)

    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            ax[i,j].set_xscale("log")
            ax[i,j].legend(loc=1)
            ax[3,j].set_xlabel("Hypocenter Distance (km)", fontsize=font)
#             ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0, which="both")
            ax[i,j].set_xlim(left=10)

    ax[0,0].set_ylabel(r"Acc. (m/s$^2$)", fontsize=font)
    ax[1,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[2,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)
    ax[3,0].set_ylabel(r"Rot. (rad/s)", fontsize=font)

    plt.show()

In [ ]:
# __makeplot_amax2(data_amax, data_amax_cosum_6, "Hdistance_km")

## Compare ADR - BSPF maximal amplitudes

In [ ]:
def __makeplot_compare_amax(data2):

    NJ = 3
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10), sharex=True)

    font = 14
    scale = 1
    msize = 12
    linew = 0.5




    ## _________________________


    ax[0,0].scatter(data2.RPFO_in_Z_adr,data2.BSPF_in_Z_adr,  s=msize, color="tab:red", label="RPFO inner Z", lw=linew, edgecolors="k", zorder=3)

    ax[0,1].scatter(data2.RPFO_in_N_adr,data2.BSPF_in_N_adr,  s=msize, color="tab:red", label="RPFO inner N", lw=linew, edgecolors="k", zorder=3)

    ax[0,2].scatter(data2.RPFO_in_E_adr, data2.BSPF_in_E_adr, s=msize, color="tab:red", label="RPFO inner E", lw=linew, edgecolors="k", zorder=3)

    ## _________________________


    ax[1,0].scatter(data2.RPFO_mi_Z_adr, data2.BSPF_mi_Z_adr, s=msize, color="tab:red", label="ADR mid Z", lw=linew, edgecolors="k", zorder=3)

    ax[1,1].scatter(data2.RPFO_mi_N_adr,data2.BSPF_mi_N_adr,  s=msize, color="tab:red", label="ADR mid N", lw=linew, edgecolors="k", zorder=3)

    ax[1,2].scatter(data2.RPFO_mi_E_adr, data2.BSPF_mi_E_adr, s=msize, color="tab:red", label="ADR mid E", lw=linew, edgecolors="k", zorder=3)


    ## _________________________


    ax[2,0].scatter(data2.RPFO_al_Z_adr, data2.BSPF_al_Z_adr, s=msize, color="tab:red", label="ADR all Z", lw=linew, edgecolors="k", zorder=3)

    ax[2,1].scatter(data2.RPFO_al_N_adr, data2.BSPF_al_N_adr, s=msize, color="tab:red", label="ADR all N", lw=linew, edgecolors="k", zorder=3)

    ax[2,2].scatter(data2.RPFO_al_E_adr, data2.BSPF_al_E_adr, s=msize, color="tab:red", label="ADR all E", lw=linew, edgecolors="k", zorder=3)



    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            ax[i,j].set_xscale("log")
            ax[2,j].set_xlabel(r"ADR (rad/s)", fontsize=font)

            ax[i,j].legend(loc=2)
            ax[i,j].grid(ls="-", color="lightgrey", alpha=0.6, zorder=0, which="both")

    for j in range(NI):
        # ax[0,j].set_title(f"0.16-16.5 Hz")
        # ax[1,j].set_title("0.03-3.3 Hz")
        # ax[2,j].set_title(f"0.02-1.3 Hz")
        ax[0,j].set_title(f"1.0-16.0 Hz")
        ax[1,j].set_title("0.5-3.0 Hz")
        ax[2,j].set_title(f"0.05- Hz")

    for i in range(NI):
        for j in range(NJ):
            ax[i,j].set_xlim(1e-11, 2e-5)
            ax[i,j].set_ylim(1e-11, 2e-5)

            ## plot one diagonal
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(1e-12, 10e-5, 1000), ls="--", c="k", zorder=2)
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(1e-8, 1e-8, 1000), ls=":", c="tab:purple", zorder=2)
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(3e-8, 3e-8, 1000), ls=":", c="tab:blue", zorder=2)

    ax[0,0].set_ylabel(r"BSPF inner (rad/s)", fontsize=font)
    ax[1,0].set_ylabel(r"BSPF mid (rad/s)", fontsize=font)
    ax[2,0].set_ylabel(r"BSPF all (rad/s)", fontsize=font)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_compare_amax(data_adr)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_amplitude_comparison_ADR_BSPF.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot_compare_amax2(data1, data2):

    NJ = 3
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10), sharex=True)

    font = 14
    scale = 1
    msize = 12
    linew = 0.5




    ## _________________________


    ax[0,0].scatter(data1.RPFO_in_Z_adr,data1.BSPF_in_Z_adr,  s=msize, color="grey", alpha=1, zorder=2)
    ax[0,0].scatter(data2.RPFO_in_Z_adr,data2.BSPF_in_Z_adr,  s=msize, color="tab:red", label="RPFO inner Z", lw=linew, edgecolors="k", zorder=3)

    ax[0,1].scatter(data1.RPFO_in_N_adr,data1.BSPF_in_N_adr,  s=msize, color="grey", alpha=1, zorder=2)
    ax[0,1].scatter(data2.RPFO_in_N_adr,data2.BSPF_in_N_adr,  s=msize, color="tab:red", label="RPFO inner N", lw=linew, edgecolors="k", zorder=3)

    ax[0,2].scatter(data1.RPFO_in_E_adr, data1.BSPF_in_E_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[0,2].scatter(data2.RPFO_in_E_adr, data2.BSPF_in_E_adr, s=msize, color="tab:red", label="RPFO inner E", lw=linew, edgecolors="k", zorder=3)

    ## _________________________


    ax[1,0].scatter(data1.RPFO_mi_Z_adr, data1.BSPF_mi_Z_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[1,0].scatter(data2.RPFO_mi_Z_adr, data2.BSPF_mi_Z_adr, s=msize, color="tab:red", label="ADR mid Z", lw=linew, edgecolors="k", zorder=3)

    ax[1,1].scatter(data1.RPFO_mi_N_adr, data1.BSPF_mi_N_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[1,1].scatter(data2.RPFO_mi_N_adr,data2.BSPF_mi_N_adr,  s=msize, color="tab:red", label="ADR mid N", lw=linew, edgecolors="k", zorder=3)

    ax[1,2].scatter(data1.RPFO_mi_E_adr, data1.BSPF_mi_E_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[1,2].scatter(data2.RPFO_mi_E_adr, data2.BSPF_mi_E_adr, s=msize, color="tab:red", label="ADR mid E", lw=linew, edgecolors="k", zorder=3)


    ## _________________________


    ax[2,0].scatter(data1.RPFO_al_Z_adr,data1.BSPF_al_Z_adr,  s=msize, color="grey", alpha=1, zorder=2)
    ax[2,0].scatter(data2.RPFO_al_Z_adr, data2.BSPF_al_Z_adr, s=msize, color="tab:red", label="ADR all Z", lw=linew, edgecolors="k", zorder=3)

    ax[2,1].scatter(data1.RPFO_al_N_adr, data1.BSPF_al_N_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[2,1].scatter(data2.RPFO_al_N_adr, data2.BSPF_al_N_adr, s=msize, color="tab:red", label="ADR all N", lw=linew, edgecolors="k", zorder=3)

    ax[2,2].scatter(data1.RPFO_al_E_adr, data1.BSPF_al_E_adr, s=msize, color="grey", alpha=1, zorder=2)
    ax[2,2].scatter(data2.RPFO_al_E_adr, data2.BSPF_al_E_adr, s=msize, color="tab:red", label="ADR all E", lw=linew, edgecolors="k", zorder=3)



    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            ax[i,j].set_xscale("log")
            ax[2,j].set_xlabel(r"ADR (rad/s)", fontsize=font)

            ax[i,j].legend(loc=2)
            ax[i,j].grid(ls="-", color="lightgrey", alpha=0.6, zorder=0, which="both")

    for j in range(NI):
        # ax[0,j].set_title(f"0.16-16.5 Hz")
        # ax[1,j].set_title("0.03-3.3 Hz")
        # ax[2,j].set_title(f"0.02-1.3 Hz")
        ax[0,j].set_title(f"1.0-16.0 Hz")
        ax[1,j].set_title("0.5-3.0 Hz")
        ax[2,j].set_title(f"0.05- Hz")

    for i in range(NI):
        for j in range(NJ):
            ax[i,j].set_xlim(1e-11, 2e-5)
            ax[i,j].set_ylim(1e-11, 2e-5)

            ## plot one diagonal
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(1e-12, 10e-5, 1000), ls="--", c="k", zorder=2)
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(1e-8, 1e-8, 1000), ls=":", c="tab:purple", zorder=2)
            ax[i,j].plot(np.linspace(1e-12, 10e-5, 1000), np.linspace(3e-8, 3e-8, 1000), ls=":", c="tab:blue", zorder=2)

    ax[0,0].set_ylabel(r"BSPF inner (rad/s)", fontsize=font)
    ax[1,0].set_ylabel(r"BSPF mid (rad/s)", fontsize=font)
    ax[2,0].set_ylabel(r"BSPF all (rad/s)", fontsize=font)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_compare_amax2(data_adr, data_adr_cosum_6)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_amplitude_comparison_ADR_BSPF_2.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

## CC values

In [ ]:
def __makeplot_cc(data1, data2, xvalue):

    NJ = 4
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15,10))

    font = 14

    scale = 1
    msize = 12
    linew = 0.1

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.BSPF_in_Z_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[0,0].scatter(data2[xvalue], data2.BSPF_in_Z_adr*scale, s=msize, color="red", label="BSPF Z", lw=linew, edgecolors="k")

    ax[0,1].scatter(data1[xvalue], data1.BSPF_in_N_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[0,1].scatter(data2[xvalue], data2.BSPF_in_N_adr*scale, s=msize, color="red", label="BSPF N", lw=linew, edgecolors="k")

    ax[0,2].scatter(data1[xvalue], data1.BSPF_in_E_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[0,2].scatter(data2[xvalue], data2.BSPF_in_E_adr*scale, s=msize, color="red", label="BSPF E", lw=linew, edgecolors="k")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_adr*scale, s=msize, color="red", label="ADR all Z", lw=linew, edgecolors="k")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_adr*scale, s=msize, color="red", label="ADR all N", lw=linew, edgecolors="k")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_adr*scale, s=msize, color="red", label="ADR all E", lw=linew, edgecolors="k")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_mi_Z_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[2,0].scatter(data2[xvalue], data2.RPFO_mi_Z_adr*scale, s=msize, color="red", label="ADR mid Z", lw=linew, edgecolors="k")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_mi_N_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[2,1].scatter(data2[xvalue], data2.RPFO_mi_N_adr*scale, s=msize, color="red", label="ADR mid N", lw=linew, edgecolors="k")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_mi_E_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[2,2].scatter(data2[xvalue], data2.RPFO_mi_E_adr*scale, s=msize, color="red", label="ADR mid E", lw=linew, edgecolors="k")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.RPFO_in_Z_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[3,0].scatter(data2[xvalue], data2.RPFO_in_Z_adr*scale, s=msize, color="red", label="ADR inner Z", lw=linew, edgecolors="k")

    ax[3,1].scatter(data1[xvalue], data1.RPFO_in_N_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[3,1].scatter(data2[xvalue], data2.RPFO_in_N_adr*scale, s=msize, color="red", label="ADR inner N", lw=linew, edgecolors="k")

    ax[3,2].scatter(data1[xvalue], data1.RPFO_in_E_adr*scale, s=msize, color="grey", alpha=0.7)
    ax[3,2].scatter(data2[xvalue], data2.RPFO_in_E_adr*scale, s=msize, color="red", label="ADR inner E", lw=linew, edgecolors="k")



    for i in range(NJ):
        for j in range(NI):
            # ax[i,j].set_yscale("log")
            ax[i,j].set_ylim(-1.1, 1.1)
            ax[i,j].legend(loc=4)
            ax[3,j].set_xlabel(xvalue, fontsize=font)
            ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[0,0].set_ylabel(r"Rot. CC", fontsize=font)
    ax[1,0].set_ylabel(r"Rot. CC", fontsize=font)
    ax[2,0].set_ylabel(r"Rot. CC", fontsize=font)
    ax[3,0].set_ylabel(r"Rot. CC", fontsize=font)

    plt.show()
    return fig

In [ ]:
# fig = __makeplot_cc(data_adr_cc, data_adr_cc_cosum_6, "Magnitude")

# fig.savefig(config['path_to_figs']+f"BSPF_statistics_cc_mag.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot_cc2(data1):

    NJ = 4
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15, 10))

    font = 14

    msize = 12
    linew = 0.5

    cmap = plt.get_cmap("seismic")

    ## _________________________

    cax = ax[0,0].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.BSPF_in_Z_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="BSPF Z", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[0,1].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.BSPF_in_N_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="BSPF N", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[0,2].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.BSPF_in_E_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="BSPF E", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    ## _________________________

    cax = ax[1,0].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_al_Z_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR all Z", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[1,1].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_al_N_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR all N", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[1,2].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_al_E_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR all E", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    ## _________________________

    cax = ax[2,0].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_mi_Z_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR mid Z", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[2,1].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_mi_N_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR mid N", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[2,2].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_mi_E_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR mid E", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    ## _________________________

    cax = ax[3,0].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_in_Z_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR inner Z", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[3,1].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_in_N_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR inner N", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)

    cax = ax[3,2].scatter(data1.Hdistance_km, data1.Magnitude, c=data1.RPFO_in_E_adr, cmap=cmap, vmin=-1, vmax=1, s=msize, label="ADR inner E", lw=linew, edgecolors="k")
    cbar = plt.colorbar(cax, pad=0.02)
    cbar.set_label(r"CC-Coefficient", fontsize=font-1)



    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_xscale("log")
            ax[i,j].set_ylim(0, 5)
            ax[i,j].legend(loc=4)
            ax[3,j].set_xlabel("Hypocentral Distance (km)", fontsize=font)
            # ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[0,0].set_ylabel(r"Magnitude", fontsize=font)
    ax[1,0].set_ylabel(r"Magnitude", fontsize=font)
    ax[2,0].set_ylabel(r"Magnitude", fontsize=font)
    ax[3,0].set_ylabel(r"Magnitude", fontsize=font)

    plt.show()
    return fig

In [ ]:
fig = __makeplot_cc2(data_adr_cc)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_cc.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

## Signal-to-Noise Ratio

In [ ]:
# pd.plotting.scatter_matrix(data_snr[['Magnitude','CoincidenceSum','BAZ','Edistance_km','Hdistance_km']], alpha=0.2, figsize=(10,10))
# plt.show();

In [ ]:
# pd.plotting.scatter_matrix(data_amax[['BSPF__Z_amax','BSPF__N_amax','BSPF__E_amax','RPFO_al_Z_amax','RPFO_al_N_amax','RPFO_al_E_amax']], alpha=0.2, figsize=(10,10))
# plt.show();

In [ ]:
def __makeplot_snr(data1, data2, xvalue):

    NJ = 5
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15, 10), sharey=False, sharex=True)

    font = 14
    scale = 1e0
    msize = 12

    plt.subplots_adjust(hspace=0.1, wspace=0.05)

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.PFO_10_Z_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[0,0].scatter(data2[xvalue], data2.PFO_10_Z_snr*scale, s=msize, zorder=2, label="PFO Z", edgecolor="k", lw=0.5, color="tab:red")

    ax[0,1].scatter(data1[xvalue], data1.PFO_10_N_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[0,1].scatter(data2[xvalue], data2.PFO_10_N_snr*scale, s=msize, zorder=2, label="PFO N", edgecolor="k", lw=0.5, color="tab:red")

    ax[0,2].scatter(data1[xvalue], data1.PFO_10_E_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[0,2].scatter(data2[xvalue], data2.PFO_10_E_snr*scale, s=msize, zorder=2, label="PFO E", edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_snr*scale, s=msize, zorder=2, label="ADR all Z", edgecolor="k", lw=0.5, color="tab:red")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_snr*scale, s=msize, zorder=2, label="ADR all N", edgecolor="k", lw=0.5, color="tab:red")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_snr*scale, s=msize, zorder=2, label="ADR all E", edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_mi_Z_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[2,0].scatter(data2[xvalue], data2.RPFO_mi_Z_snr*scale, s=msize, zorder=2, label="ADR mid Z", edgecolor="k", lw=0.5, color="tab:red")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_mi_N_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[2,1].scatter(data2[xvalue], data2.RPFO_mi_N_snr*scale, s=msize, zorder=2, label="ADR mid N", edgecolor="k", lw=0.5, color="tab:red")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_mi_E_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[2,2].scatter(data2[xvalue], data2.RPFO_mi_E_snr*scale, s=msize, zorder=2, label="ADR mid E", edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.RPFO_in_Z_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[3,0].scatter(data2[xvalue], data2.RPFO_in_Z_snr*scale, s=msize, zorder=2, label="ADR inner Z", edgecolor="k", lw=0.5, color="tab:red")

    ax[3,1].scatter(data1[xvalue], data1.RPFO_in_N_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[3,1].scatter(data2[xvalue], data2.RPFO_in_N_snr*scale, s=msize, zorder=2, label="ADR inner N", edgecolor="k", lw=0.5, color="tab:red")

    ax[3,2].scatter(data1[xvalue], data1.RPFO_in_E_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[3,2].scatter(data2[xvalue], data2.RPFO_in_E_snr*scale, s=msize, zorder=2, label="ADR inner E", edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[4,0].scatter(data1[xvalue], data1.BSPF__Z_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[4,0].scatter(data2[xvalue], data2.BSPF__Z_snr*scale, s=msize, zorder=2, label="BSPF Z", edgecolor="k", lw=0.5, color="tab:red")

    ax[4,1].scatter(data1[xvalue], data1.BSPF__N_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[4,1].scatter(data2[xvalue], data2.BSPF__N_snr*scale, s=msize, zorder=2, label="BSPF N", edgecolor="k", lw=0.5, color="tab:red")

    ax[4,2].scatter(data1[xvalue], data1.BSPF__E_snr*scale, s=msize, zorder=2, color="grey", alpha=0.7)
    ax[4,2].scatter(data2[xvalue], data2.BSPF__E_snr*scale, s=msize, zorder=2, label="BSPF E", edgecolor="k", lw=0.5, color="tab:red")


    for i in range(NJ):
        for j in range(NI):
            if i < 4:
                ax[i,j].set_yscale("log")
                ax[i,j].set_ylim(0, 1e4)

            ax[i,j].legend(loc=2)
            ax[4,j].set_xlabel(xvalue, fontsize=font)
            ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)
            ax[4,j].set_ylim(0, 30)
            if j > 0:
                ax[i,j].set_yticklabels([])

    ax[0,0].set_ylabel(r"SNR", fontsize=font)
    ax[1,0].set_ylabel(r"SNR", fontsize=font)
    ax[2,0].set_ylabel(r"SNR", fontsize=font)
    ax[3,0].set_ylabel(r"SNR", fontsize=font)
    ax[4,0].set_ylabel(r"SNR", fontsize=font)

    plt.show()
    return fig

In [ ]:
fig = __makeplot_snr(data_snr, data_snr_cosum_6, "Magnitude")

fig.savefig(config['path_to_figs']+f"BSPF_statistics_snr_magnitude.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

In [ ]:
def __makeplot_snr_colored(data1, data2, xvalue):

    NJ = 5
    NI = 3

    fig, ax = plt.subplots(NJ, NI, figsize=(15, 10), sharey=True, sharex=True)

    font = 14
    scale = 1e0
    msize = 12

    plt.subplots_adjust(hspace=0.1, wspace=0.05)

    import matplotlib
    cmap = matplotlib.colors.ListedColormap(["tab:red", "tab:orange", "tab:green"])

    ## _________________________

    ax[0,0].scatter(data1[xvalue], data1.PFO_10_Z_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="PFO Z")
    # ax[0,0].scatter(data2[xvalue], data2.PFO_10_Z_snr*scale, s=msize, zorder=2, c=data2.CoincidenceSum, edgecolor="k", lw=0.5)

    ax[0,1].scatter(data1[xvalue], data1.PFO_10_N_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="PFO N")
    # ax[0,1].scatter(data2[xvalue], data2.PFO_10_N_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[0,2].scatter(data1[xvalue], data1.PFO_10_E_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="PFO E")
    # ax[0,2].scatter(data2[xvalue], data2.PFO_10_E_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[1,0].scatter(data1[xvalue], data1.RPFO_al_Z_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR all Z")
    # ax[1,0].scatter(data2[xvalue], data2.RPFO_al_Z_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[1,1].scatter(data1[xvalue], data1.RPFO_al_N_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR all N")
    # ax[1,1].scatter(data2[xvalue], data2.RPFO_al_N_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[1,2].scatter(data1[xvalue], data1.RPFO_al_E_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR all E")
    # ax[1,2].scatter(data2[xvalue], data2.RPFO_al_E_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[2,0].scatter(data1[xvalue], data1.RPFO_mi_Z_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR mid Z")
    # ax[2,0].scatter(data2[xvalue], data2.RPFO_mi_Z_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[2,1].scatter(data1[xvalue], data1.RPFO_mi_N_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR mid N")
    # ax[2,1].scatter(data2[xvalue], data2.RPFO_mi_N_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[2,2].scatter(data1[xvalue], data1.RPFO_mi_E_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR mid E")
    # ax[2,2].scatter(data2[xvalue], data2.RPFO_mi_E_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[3,0].scatter(data1[xvalue], data1.RPFO_in_Z_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR inner Z")
    # ax[3,0].scatter(data2[xvalue], data2.RPFO_in_Z_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[3,1].scatter(data1[xvalue], data1.RPFO_in_N_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR inner N")
    # ax[3,1].scatter(data2[xvalue], data2.RPFO_in_N_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[3,2].scatter(data1[xvalue], data1.RPFO_in_E_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="ADR inner E")
    # ax[3,2].scatter(data2[xvalue], data2.RPFO_in_E_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ## _________________________

    ax[4,0].scatter(data1[xvalue], data1.BSPF__Z_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="BSPF Z")
    # ax[4,0].scatter(data2[xvalue], data2.BSPF__Z_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[4,1].scatter(data1[xvalue], data1.BSPF__N_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="BSPF N")
    # ax[4,1].scatter(data2[xvalue], data2.BSPF__N_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")

    ax[4,2].scatter(data1[xvalue], data1.BSPF__E_snr*scale, s=msize, zorder=2, c=data1.CoincidenceSum, cmap=cmap, alpha=0.7, label="BSPF E")
    # ax[4,2].scatter(data2[xvalue], data2.BSPF__E_snr*scale, s=msize, zorder=2, edgecolor="k", lw=0.5, color="tab:red")


    for i in range(NJ):
        for j in range(NI):
            ax[i,j].set_yscale("log")
            # ax[i,j].set_xscale("log")
            ax[i,j].legend(loc=2)
            ax[4,j].set_xlabel(xvalue, fontsize=font)
            ax[i,j].set_ylim(9e-1, 1e4)
            ax[i,j].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[0,0].set_ylabel(r"SNR", fontsize=font)
    ax[1,0].set_ylabel(r"SNR", fontsize=font)
    ax[2,0].set_ylabel(r"SNR", fontsize=font)
    ax[3,0].set_ylabel(r"SNR", fontsize=font)
    ax[4,0].set_ylabel(r"SNR", fontsize=font)

    plt.show()
    return fig

In [ ]:
fig = __makeplot_snr_colored(data_snr, data_snr_cosum_6, "Magnitude")

fig.savefig(config['path_to_figs']+f"BSPF_statistics_snr_magnitude_colored.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

## Velocity

In [ ]:
from functions.compute_linear_regression import __compute_linear_regression

In [ ]:
def __makeplot_velocity(data):

    from numpy import array, log10

    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    font = 14

    scale = 1e3
    msize = 10
    linew = 0.3

    data0 = data.dropna()

    x_arr1 = data0.BSPF__Z_amax*scale
    y_arr1= 0.5*data0.PFO_10_T_amax*scale

    xx1 = array([b for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])
    yy1 = array([a for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])

#     xx1, yy1 = log10(xx1), log10(yy1)

    x_axis1 = np.linspace(0, xx1.max(), 100)


    ## linear regression
#     b1, a1 = __compute_orthogonal_distance_regression(xx1, yy1, xerr=None, yerr=None, bx=None, by=None)
    b1, a1 = __compute_linear_regression(xx1, yy1, intercept_is_zero=True)

    ax[0].scatter(xx1, yy1, s=20, c='tab:red', marker="o", label=f"N={len(xx1)}", zorder=3, lw=linew, edgecolors="k")
    ax[0].plot(x_axis1, a1+b1*x_axis1, color='k', ls="--", label=f"f(x) = {a1:.0f} + {b1:.0f} x", zorder=1)

    ax[0].legend(loc=2, fontsize=font)
    ax[0].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[0].set_xlim(left=0)
    ax[0].set_ylim(bottom=0)

    ax[0].set_xlabel("vertical rotation rate $\Omega_Z$ (mrad/s)", fontsize=font)
    ax[0].set_ylabel("transverse acceleration $a_T$ (mm/s$^2$)", fontsize=font)
    ax[0].set_title("Love Phase Velocity", fontsize=font+1)


    ## ______________________________________________

    x_arr2 = data0.BSPF__T_amax*scale
    y_arr2= data0.PFO_10_Z_amax*scale

    xx2 = array([b for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])
    yy2 = array([a for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])

#     xx2, yy2 = log10(xx2), log10(yy2)

    x_axis2 = np.linspace(0, xx2.max(), 100)

    ## linear regression
#     b2, a2 = __compute_orthogonal_distance_regression(xx2, yy2, xerr=None, yerr=None, bx=None, by=None)
    b2, a2 = __compute_linear_regression(xx2, yy2, intercept_is_zero=True)

    ax[1].scatter(xx2, yy2, s=20, c='tab:red' ,marker="o", label=f"N={len(xx2)}", zorder=3, lw=linew, edgecolors="k")
    ax[1].plot(x_axis2, a2+b2*x_axis2, color='k', ls="--", label=f"f(x) = {a2:.0f} + {b2:.0f} x", zorder=1)

    ax[1].legend(loc=2, fontsize=font)
    ax[1].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[1].set_xlim(left=0)
    ax[1].set_ylim(bottom=0)

    ax[1].set_xlabel("transverse rotation rate $\Omega_T$ (mrad/s)", fontsize=font)
    ax[1].set_ylabel("vertical acceleration $a_Z$ (mm/s$^2$)", fontsize=font)
    ax[1].set_title("Rayleigh Phase Velocity", fontsize=font+1)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_velocity(data_amax)

fig.savefig(config['path_to_figs']+f"BSPF_statistics_velocity_from_Amax.png", dpi=200, bbox_inches='tight', pad_inches=0.05);

In [ ]:
fig = __makeplot_velocity(data_amax[data_amax.Magnitude < 3])

fig.savefig(config['path_to_figs']+f"BSPF_statistics_velocity_from_Amax_M<3.png", dpi=200, bbox_inches='tight', pad_inches=0.05);

In [ ]:
def __makeplot_velocity_multiple(data, odr=False):

    from numpy import array, log10

    def __regress1(data, scale):
        data0 = data.dropna()

        x_arr1 = data0.BSPF__Z_amax*scale
        y_arr1= 0.5*data0.PFO_10_T_amax*scale

        xx1 = array([b for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])
        yy1 = array([a for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])

        x_axis1 = np.linspace(0, xx1.max(), 100)

        ## linear regression
        if odr:
            b1, a1 = __compute_orthogonal_distance_regression(xx1, yy1, xerr=None, yerr=None, bx=None, by=None)
        else:
            b1, a1 = __compute_linear_regression(xx1, yy1, intercept_is_zero=True)

        return x_axis1, b1, a1

    scale = 1e3

    data0 = data.dropna()

    x_arr1 = data0.BSPF__Z_amax*scale
    y_arr1= 0.5*data0.PFO_10_T_amax*scale

    xx1 = array([b for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])
    yy1 = array([a for a, b in zip(y_arr1, x_arr1) if a/b > 1e-3])


    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    font = 14

    msize = 10
    linew = 0.3

    ax[0].scatter(xx1, yy1, s=20, c='tab:red', marker="o", label=f"N={len(xx1)}", zorder=3, lw=linew, edgecolors="k")

    x_axis1, b1, a1 = __regress1(data[data.Magnitude > 0], scale)
    ax[0].plot(x_axis1, a1+b1*x_axis1, color='k', ls="--", label=f"f(x) = {a1:.0f} + {b1:.0f} x  (M>0)", zorder=1)

    x_axis1, b1, a1 = __regress1(data[data.Magnitude < 3], scale)
    ax[0].plot(x_axis1, a1+b1*x_axis1, color='b', ls="--", label=f"f(x) = {a1:.0f} + {b1:.0f} x  (M<3)", zorder=1)

    ax[0].legend(loc=2, fontsize=font)
    ax[0].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[0].set_xlim(left=0)
    ax[0].set_ylim(bottom=0)

    ax[0].set_xlabel("vertical rotation rate $\Omega_Z$ (mrad/s)", fontsize=font)
    ax[0].set_ylabel("transverse acceleration $a_T$ (mm/s$^2$)", fontsize=font)
    ax[0].set_title("Love Phase Velocity", fontsize=font+1)


    ## ______________________________________________

    def __regress2(data, scale):
        data0 = data.dropna()

        x_arr2 = data0.BSPF__T_amax*scale
        y_arr2= data0.PFO_10_Z_amax*scale

        xx2 = array([b for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])
        yy2 = array([a for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])

        x_axis2 = np.linspace(0, xx2.max(), 100)

        ## linear regression
        if odr:
            b2, a2 = __compute_orthogonal_distance_regression(xx2, yy2, xerr=None, yerr=None, bx=None, by=None)
        else:
            b2, a2 = __compute_linear_regression(xx2, yy2, intercept_is_zero=True)

        return x_axis2, b2, a2 

    x_arr2 = data0.BSPF__T_amax*scale
    y_arr2= data0.PFO_10_Z_amax*scale

    xx2 = array([b for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])
    yy2 = array([a for a, b in zip(y_arr2, x_arr2) if a/b > 1e-3])

    x_axis2 = np.linspace(0, xx2.max(), 100)


    ax[1].scatter(xx2, yy2, s=20, c='tab:red' ,marker="o", label=f"N={len(xx2)}", zorder=3, lw=linew, edgecolors="k")

    x_axis2, b2, a2 = __regress2(data[data.Magnitude > 0], scale)
    ax[1].plot(x_axis2, a2+b2*x_axis2, color='k', ls="--", label=f"f(x) = {a2:.0f} + {b2:.0f} x", zorder=1)

    x_axis2, b2, a2 = __regress2(data[data.Magnitude < 3], scale)
    ax[1].plot(x_axis2, a2+b2*x_axis2, color='b', ls="--", label=f"f(x) = {a2:.0f} + {b2:.0f} x", zorder=1)


    ax[1].legend(loc=2, fontsize=font)
    ax[1].grid(ls="--", color="grey", alpha=0.6, zorder=0)

    ax[1].set_xlim(left=0)
    ax[1].set_ylim(bottom=0)

    ax[1].set_xlabel("transverse rotation rate $\Omega_T$ (mrad/s)", fontsize=font)
    ax[1].set_ylabel("vertical acceleration $a_Z$ (mm/s$^2$)", fontsize=font)
    ax[1].set_title("Rayleigh Phase Velocity", fontsize=font+1)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_velocity_multiple(data_amax_cosum_6, odr=True);

fig.savefig(config['path_to_figs']+f"BSPF_statistics_velocity_multiple_lin.png", dpi=200, bbox_inches='tight', pad_inches=0.05);